# The task

You need to create a prototype pricing model that can go through further validation and testing before being put into production. Eventually, this model may be the basis for fully automated quoting to clients, but for now, the desk will use it with manual oversight to explore options with the client. 

You should write a function that is able to use the data you created previously to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:

1. Injection dates. 
2. Withdrawal dates.
3. The prices at which the commodity can be purchased/sold on those dates.
4. The rate at which the gas can be injected/withdrawn.
5. The maximum volume that can be stored.
6. Storage costs.

Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero. Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs.

# Take the model from task 1

First we take the model built in task 1 so that we can estimate the price of natual gas at any given date that is not a weekend or holiday.

In [1]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression

In [2]:
price = pd.read_csv("./Nat_Gas.csv")
price['Dates'] = pd.to_datetime(price['Dates'], format="%m/%d/%y")
price = price.set_index("Dates")
result = seasonal_decompose(price, model='additive', extrapolate_trend='freq')

# train test split function
def train_test_split(df, test_size):
    return df[0:-test_size], df[-test_size:]

In [3]:
# model for trend
df_trend = pd.DataFrame(result.trend)
df_trend.rename(columns={'trend': 'Prices'}, inplace=True)

# use the number of days from the first date in the data as predictor
df_trend['n_days'] = [(d - df_trend.index[0]).days for d in df_trend.index]

# train test split
train_trend, test_trend = train_test_split(df_trend, 12)
X_train_trend = train_trend['n_days'].values.reshape(-1, 1)
y_train_trend = train_trend['Prices'].values.reshape(-1, 1)

# linear regression
model_trend = LinearRegression()
model_trend.fit(X_train_trend, y_train_trend)

# model for seasonality
df_seasonal = pd.DataFrame(result.seasonal)
df_seasonal.rename(columns={'seasonal': 'Prices'}, inplace=True)

# use the number of days from the first date in the data as predictor
df_seasonal['n_days'] = [(d - df_seasonal.index[0]).days for d in df_seasonal.index]

# train test split
train_seasonal, test_seasonal = train_test_split(df_seasonal, 12)
X_train_seasonal = train_seasonal['n_days'].values
y_train_seasonal = train_seasonal['Prices'].values

# nonlinear model fit using the sine function
import numpy as np
from scipy.optimize import curve_fit
def model_seasonal(x, a, b, c):
    return a * np.sin(b * x + c)

# the initial guessing of the parameters p0 is important
params, _ = curve_fit(model_seasonal, X_train_seasonal, y_train_seasonal, p0=[0.5, 2*np.pi/365, 0])

# Estimated price for any date

I will use the combined tread and seasonality model to estimate the natural gas price for any date since the first date in the dataset.

In [4]:
from datetime import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
calendar = USFederalHolidayCalendar()
holidays = calendar.holidays(start='2023-01-01', end='2026-12-31')

# Combine the models for trend and seasonality
def get_price(any_date: str) -> float:
    '''
    params:
        any_date (str): input date string in the format of 'YYYY-MM-DD'
    
    return:
        The natural gas' price on that date in float. If the input date is a weekend 
        or holiday, return -1.
    '''
    date = datetime.strptime(any_date, '%Y-%m-%d')
    # check if it is a weekend or holiday
    if date.weekday() >= 5 or date in holidays:
        return -1
    # if the date is in the original dataset, use the original data
    elif date in price.index:
        return price.loc[date].Prices
    else:
        n_days = np.array([(date - price.index[0]).days])
        estimation = model_trend.predict(n_days.reshape(-1, 1)).reshape(len(n_days)) + \
                        model_seasonal(n_days, params[0], params[1], params[2])
        return round(estimation[0], 1)

In [5]:
from typing import Dict
def contract(injection: Dict, withdrawal: Dict, max_vol: float, storage_cost: float, transport_cost: float) -> float:
    '''
    Calculate the contract price using the input parameters.
    
    params:
        injection (Dict):     A dictionary that contains dates ('YYYY-MM-DD'), rate (million MMBtu per day), 
                                cost (dollars amount per million MMBtu)
        withdrawal (Dict):    A dictionary that contains dates ('YYYY-MM-DD'), rate (million MMBtu per day), 
                                cost (dollars amount per million MMBtu)
        max_vol (float):      Maximum volumn of the storage facility (million MMBtu)
        storage_cost (float): dollars amount per day
        transport_cost (float): dollars amount per to/from the storage facility
    
    return:
        dollars of the contract price if profitable, otherwise return a message saying not profitable.
    '''
    # assume the storage facility is empty before the first injection;
    # the client is sensible: always buy at low-price seasons and sell at high-price seasons;
    # to calculate storage cost, calculate the number of days between the withdrawal date
    # that empties the storage and the first injection date, and then re-count from the next injection date;
    # to be fair, we will share the profit with the client 50/50
    total_vol = 0
    total_purchase_cost = 0
    total_injection_cost = 0
    total_withdrawal_income = 0
    total_withdrawal_cost = 0
    total_transport_cost = 0
    total_storage_cost = 0
    all_dates = sorted(injection['dates'] + withdrawal['dates'])
    injection_in_holiday = []
    withdrawal_in_holiday = []
    for date in all_dates:
        price = get_price(date)
        if date in injection['dates']:
            if max_vol - total_vol >= injection['rate'] and price > -1:
                if total_vol == 0:
                    # first injection date of this round
                    first_injection_date = datetime.strptime(date, '%Y-%m-%d')
                total_vol += injection['rate']
                total_purchase_cost += price * injection['rate'] * 1e6
                total_injection_cost += injection['cost'] * injection['rate']
                total_transport_cost += transport_cost
            elif max_vol > total_vol and max_vol - total_vol < injection['rate'] and price > -1:
                total_vol = max_vol
                total_purchase_cost += price * (max_vol - total_vol) * 1e6
                total_injection_cost += injection['cost'] * (max_vol - total_vol)
                total_transport_cost += transport_cost
            elif price == -1:
                injection_in_holiday.append(date)
        elif date in withdrawal['dates']:
            if total_vol >= withdrawal['rate'] and price > -1:
                total_vol -= withdrawal['rate']
                if total_vol == 0:
                    last_withdrawal_date = datetime.strptime(date, '%Y-%m-%d')
                    total_storage_cost += storage_cost * (last_withdrawal_date - first_injection_date).days
                total_withdrawal_income += price * withdrawal['rate'] * 1e6
                total_withdrawal_cost += withdrawal['cost'] * withdrawal['rate']
                total_transport_cost += transport_cost
            elif total_vol > 0 and total_vol < withdrawal['rate'] and price > -1:
                total_vol = 0
                last_withdrawal_date = datetime.strptime(date, '%Y-%m-%d')
                total_storage_cost += storage_cost * (last_withdrawal_date - first_injection_date).days
                total_withdrawal_income += price * total_vol * 1e6
                total_withdrawal_cost += withdrawal['cost'] * total_vol
                total_transport_cost += transport_cost
            elif price == -1:
                withdrawal_in_holiday.append(date)
    contract_price = (total_withdrawal_income - total_purchase_cost - total_injection_cost - total_withdrawal_cost - 
                        total_transport_cost - total_storage_cost) / 2
    print(f"Total purchase cost: {total_purchase_cost}")
    print(f"Total storage cost: {total_storage_cost}")
    print(f"Total transport cost: {total_transport_cost}")
    print(f"Total withdrawal cost: {total_withdrawal_cost}")
    print(f"Total injection cost: {total_injection_cost}")
    print(f"Total selling income: {total_withdrawal_income}")
    if len(injection_in_holiday) > 0:
        print(f"These injection dates are either in weekends or holidays: {', '.join(injection_in_holiday)}." + 
             f" Please take them out from contract.")
    if len(withdrawal_in_holiday) > 0:
        print(f"These withdrawal dates are either in weekends or holidays: {', '.join(withdrawal_in_holiday)}." + 
             f" Please take them out from contract.")
    if contract_price > 0:
        return contract_price
    else:
        print("Not profitable probably because some injection/withdrawal dates are in holidays.")

In [6]:
# test 1
injection = {
    'dates': ['2023-06-27', '2023-06-28', '2023-06-30'],
    'rate': 1, # million MMBtu per day 
    'cost': 10000 # dollars per million MMBtu
}
withdrawal = {
    'dates': ['2023-11-30', '2023-12-01', '2023-12-04'],
    'rate': 1, # million MMBtu per day
    'cost': 10000 # dollars per million MMBtu
}
max_vol = 10 # million MMBtu
storage_cost = 3300 # dollars per day
transport_cost = 50000 # dollars per transportation to / from storage facility
contract(injection, withdrawal, max_vol, storage_cost, transport_cost)

Total purchase cost: 32900000.0
Total storage cost: 528000
Total transport cost: 300000
Total withdrawal cost: 30000
Total injection cost: 30000
Total selling income: 36600000.0


1406000.0

In [7]:
# test 2
injection = {
    'dates': ['2023-06-27', '2023-06-28', '2023-06-30', '2023-07-01'],
    'rate': 1, # million MMBtu per day 
    'cost': 10000 # dollars per million MMBtu
}
withdrawal = {
    'dates': ['2023-11-30', '2023-12-01', '2023-12-02', '2023-12-03'],
    'rate': 1, # million MMBtu per day
    'cost': 10000 # dollars per million MMBtu
}
max_vol = 10 # million MMBtu
storage_cost = 3300 # dollars per day
transport_cost = 50000 # dollars per transportation to / from storage facility
contract(injection, withdrawal, max_vol, storage_cost, transport_cost)

Total purchase cost: 32900000.0
Total storage cost: 0
Total transport cost: 250000
Total withdrawal cost: 20000
Total injection cost: 30000
Total selling income: 24400000.0
These injection dates are either in weekends or holidays: 2023-07-01. Please take them out from contract.
These withdrawal dates are either in weekends or holidays: 2023-12-02, 2023-12-03. Please take them out from contract.
Not profitable probably because some injection/withdrawal dates are in holidays.


In [8]:
# test 3
injection = {
    'dates': ['2023-06-27', '2023-06-28', '2023-06-30', '2023-07-01', '2023-07-02'],
    'rate': 1, # million MMBtu per day 
    'cost': 10000 # dollars per million MMBtu
}
withdrawal = {
    'dates': ['2023-11-30', '2023-12-01', '2023-12-02', '2023-12-03', '2023-12-04'],
    'rate': 1, # million MMBtu per day
    'cost': 10000 # dollars per million MMBtu
}
max_vol = 10 # million MMBtu
storage_cost = 3300 # dollars per day
transport_cost = 50000 # dollars per transportation to / from storage facility
contract(injection, withdrawal, max_vol, storage_cost, transport_cost)

Total purchase cost: 32900000.0
Total storage cost: 528000
Total transport cost: 300000
Total withdrawal cost: 30000
Total injection cost: 30000
Total selling income: 36600000.0
These injection dates are either in weekends or holidays: 2023-07-01, 2023-07-02. Please take them out from contract.
These withdrawal dates are either in weekends or holidays: 2023-12-02, 2023-12-03. Please take them out from contract.


1406000.0